In [ ]:
"""
Purpose: To test to see if theory on how can distinguish incorrect splits 
from correct splits using webbing, widht and branch angle

Ideas: if looked at the splits from a cytoskeelton viewpoint
1) if split off into equal width portions at high sibling
angle then will see webbing
2) If not have high sibling angle then might not need 

BUT USUALLY DON'T SEE LOW SIBLING ANGLES WITH THE SMALL AXON
WHICH IS WHY SEE WEBBING THERE

Things would be able to need:
1) Visualize the error and non-error branches of a cell
- border branches and downstream errors

2) Function that goes through and applies a new rule to a cell
a) Highlights the branches that will be cut according to new rule
b) Measure precision/recall on branch cuts
c) Measure overall cell proofreading precision/recall (branch wise or synaptically)

3) Might need to develop a new function that moves along graph and can look at:
- skeleton/parent angle

Plan of Attack:
1) 

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-04-13 18:00:23,252 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-13 18:00:23,254 - settings - Setting database.user to celiib
INFO - 2021-04-13 18:00:23,255 - settings - Setting database.password to newceliipass
INFO - 2021-04-13 18:00:23,312 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-04-13 18:00:23,314 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-13 18:00:23,332 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-04-13 18:00:23,755 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-13 18:00:23,795 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-13 18:00:23,797 - settings - Setting database.user to celiib
INFO - 2021-04-13 18:00:23,798 - settings - Setting database.password to newceliipass
INFO - 2021-04-13 18:00:23,799 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-13 18:00:23,800 - settings - Setting database.user to celiib
INFO - 2021-04-13 18:00:23,800 - settings - Setting database.password to newceliipass
INFO - 2021-04-13 18:00:23,803 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-13 18:00:24,248 - settings - Setting enable_python_native_blobs to True


In [3]:
#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-04-13 18:00:24,383 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-13 18:00:24,385 - settings - Setting database.user to celiib
INFO - 2021-04-13 18:00:24,386 - settings - Setting database.password to newceliipass
INFO - 2021-04-13 18:00:24,390 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-13 18:00:24,885 - settings - Setting enable_python_native_blobs to True


In [10]:
import neuron_utils as nru
import neuron_visualizations as nviz

# Fetching the Neuron

In [4]:
minnie.AutoProofreadValidationBorder()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version the version of code used for this cell typing classification,axon_version,parent_idx,web_size_faces,web_size_volume,web_size_skeleton,web_size_ray_trace_percentile,web_bbox_ratios_max,web_bbox_ratios_min,web_volume_ratio,web_cdf,parent_n_large_boutons,parent_n_boutons,parent_no_bouton_median,parent_no_spine_median_mesh_center,child_no_bouton_median_min,child_no_bouton_median_diff_min,child_no_spine_median_mesh_center_min,child_no_spine_median_mesh_center_diff_min,child_angle_min,child_n_boutons_min,child_n_large_boutons_min,child_no_bouton_median_max,child_no_bouton_median_diff_max,child_no_spine_median_mesh_center_max,child_no_spine_median_mesh_center_diff_max,child_angle_max,child_n_boutons_max,child_n_large_boutons_max,sibling_angles_min,sibling_angles_max,label
864691135113167769,0,0.25,30.00,5,0,0,3,0,25,124,124043806.52500916,1543.817733468448,329.3614638698137,1.9818033758876452,1.1456226962516913,2.388934917594883,0.742895,3,15,106.27459096347302,110.75343601224178,59.20002831337389,-47.074562650099125,59.20002831337389,-51.553407698867886,18.34,0,0,59.20002831337389,-47.074562650099125,59.20002831337389,-51.553407698867886,114.46,10,0,101.18,101.18,valid
864691135113167769,0,0.25,30.00,5,0,0,3,0,33,362,1158687079.9782996,3674.7854580103326,372.5871005673645,3.509178213022797,1.3529679330298574,1.963608699589879,0.5540505,0,0,182.76536725111384,182.76536725111384,92.4871982140348,-90.27816903707904,104.68539546757829,-78.07997178353556,10.95,1,0,92.4871982140348,-90.27816903707904,104.68539546757829,-78.07997178353556,38.11,15,3,48.74,48.74,valid
864691135113167769,0,0.25,30.00,5,0,0,3,0,35,51,26370949.94332186,1019.6161407995952,176.78855354532072,2.2322996232226795,1.2516471421494915,1.901370012853932,0.303688,2,17,102.08743604032234,112.09276481057641,100.48239021999521,-1.6050458203271347,102.12305340650975,-9.969711404066658,23.74,0,0,100.48239021999521,-1.6050458203271347,102.12305340650975,-9.969711404066658,126.54,1,1,149.17,149.17,invalid
864691135113167769,0,0.25,30.00,5,0,0,3,0,58,123,170487768.338267,2240.5780127218754,198.84433280695026,1.7905620799410142,1.0344524389415621,2.3329946927315195,0.660586,0,3,84.28187368323856,91.48739940492703,49.558819053315645,-34.72305462992291,53.31209769981907,-38.17530170510796,25.87,4,0,49.558819053315645,-34.72305462992291,53.31209769981907,-38.17530170510796,43.92,16,2,63.48,63.48,valid
864691135113167769,0,0.25,30.00,5,0,0,3,0,64,40,47594919.924835205,937.009081606932,251.63548827125797,1.1646130862813708,1.044847594099291,2.271011784744647,0.5999935000000001,0,1,108.18690772465524,112.71485462886615,107.87814494638877,-0.3087627782664697,116.33563389870751,3.620779269841364,38.63,2,0,107.87814494638877,-0.3087627782664697,116.33563389870751,3.620779269841364,55.95,3,0,92.78,92.78,valid
864691135113167769,0,0.25,30.00,5,0,0,3,0,65,104,218020349.15270233,2098.1580232694077,313.20655984441794,1.1392826708250756,1.047736009406659,1.9597572689690155,0.6987399999999999,0,2,107.87814494638877,116.33563389870751,52.318986773229035,-55.55915817315974,54.367293722829736,-61.96834017587778,4.34,1,0,52.318986773229035,-55.55915817315974,54.367293722829736,-61.96834017587778,29.16,6,0,32.55,32.55,valid
864691135113167769,0,0.25,30.00,5,0,0,3,0,66,200,398468976.0860605,2053.0175329991107,389.6132826512314,1.5289558965742194,1.2059730074994737,2.745894503130506,0.7163029999999999,0,0,101.32018886008184,101.32018886008184,120.84307575495245,19.52288689487061,120.84307575495245,19.52288689487061,4

In [5]:
minnie.AutoProofreadValidationBorderNeurons()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version the version of code used for this cell typing classification,decomposition
864691135925564174,0,0.25,30.00,5,0,0,3,=BLOB=
864691136008425132,0,0.25,30.00,5,0,0,3,=BLOB=
864691135393964789,0,0.25,30.00,5,0,0,3,=BLOB=
864691136333790899,0,0.25,30.00,5,0,0,3,=BLOB=
864691135974454895,0,0.25,30.00,5,0,0,3,=BLOB=
864691136099807093,0,0.25,30.00,5,0,0,3,=BLOB=
864691136333776819,0,0.25,30.00,5,0,0,3,=BLOB=
864691136311740477,0,0.25,30.00,5,0,0,3,=BLOB=
864691136008573614,0,0.25,30.00,5,0,0,3,=BLOB=
864691135974454383,0,0.25,30.00,5,0,0,3,=BLOB=


In [17]:
segment_id = 864691136008573614
segment_id = 864691135212632448
minnie.AutoProofreadValidationBorder() & dict(segment_id=segment_id)

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version the version of code used for this cell typing classification,axon_version,parent_idx,web_size_faces,web_size_volume,web_size_skeleton,web_size_ray_trace_percentile,web_bbox_ratios_max,web_bbox_ratios_min,web_volume_ratio,web_cdf,parent_n_large_boutons,parent_n_boutons,parent_no_bouton_median,parent_no_spine_median_mesh_center,child_no_bouton_median_min,child_no_bouton_median_diff_min,child_no_spine_median_mesh_center_min,child_no_spine_median_mesh_center_diff_min,child_angle_min,child_n_boutons_min,child_n_large_boutons_min,child_no_bouton_median_max,child_no_bouton_median_diff_max,child_no_spine_median_mesh_center_max,child_no_spine_median_mesh_center_diff_max,child_angle_max,child_n_boutons_max,child_n_large_boutons_max,sibling_angles_min,sibling_angles_max,label
864691135212632448,0,0.25,30.00,5,0,0,3,0,28,133,390535919.83182776,1565.9395585198358,651.1394877031369,1.193511706248254,1.0435312459619979,1.9164638613523934,0.742695,0,0,254.7941811047929,254.7941811047929,102.08411387815694,-152.71006722663594,102.08411387815694,-152.71006722663594,32.99,0,0,102.08411387815694,-152.71006722663594,102.08411387815694,-152.71006722663594,63.57,0,0,78.4,78.4,valid
864691135212632448,0,0.25,30.00,5,0,0,3,0,30,193,577043050.5725784,4237.103459205819,117.26081408142988,1.8196341763068717,1.0001060659622458,2.815815302017368,0.11546,0,0,72.34132489447025,72.34132489447025,199.3733577256494,127.03203283117917,199.3733577256494,127.03203283117917,42.36,0,0,199.3733577256494,127.03203283117917,199.3733577256494,127.03203283117917,164.48,2,1,88.18,144.46,valid
864691135212632448,0,0.25,30.00,5,0,0,3,0,33,47,50816922.12428792,1394.4838097757017,162.0120269374509,2.001593026164973,1.1280692821127332,2.35418820928229,0.744082,0,3,55.487321773305815,56.38619819851696,62.33416821102783,6.846846437722014,62.33416821102783,5.947970012510872,13.91,0,0,62.33416821102783,6.846846437722014,62.33416821102783,5.947970012510872,109.39,0,0,122.87,122.87,valid
864691135212632448,0,0.25,30.00,5,0,0,3,0,35,93,143702916.1653439,1725.6236448111838,211.4361746516355,2.334004723393049,1.1426624425432275,2.470780865280611,0.317905,0,2,43.59058779443179,60.74929878398849,71.0792718236183,27.48868402918651,42.91526733380065,-17.834031450187844,32.24,0,0,71.0792718236183,27.48868402918651,42.91526733380065,-17.834031450187844,132.35,5,2,30.14,148.73,valid
864691135212632448,0,0.25,30.00,5,0,0,3,0,36,228,723767405.6366997,4307.288726601057,93.54971007278758,1.5080748633374337,1.051275208285667,3.1518211364347604,0.09288165,1,7,102.86489766253152,108.27691904934231,55.777529882184,-47.08736778034752,62.47924403071497,-45.79767501862734,29.36,1,0,55.777529882184,-47.08736778034752,62.47924403071497,-45.79767501862734,81.96,2,1,92.79,146.24,invalid
864691135212632448,0,0.25,30.00,5,0,0,3,0,38,89,112322042.73697837,1523.656200330534,249.76281392429422,1.1570277375802465,1.0222445648404326,2.6825455361787274,0.798753,0,2,55.777529882184,62.47924403071497,61.84313770780982,6.065607825625818,65.88805220796088,3.408808177245909,5.69,0,0,61.84313770780982,6.065607825625818,65.88805220796088,3.408808177245909,82.56,4,1,85.45,85.45,invalid
864691135212632448,0,0.25,30.00,5,0,0,3,0,40,137,380879232.0943247,3272.6563470490432,90.82152302126617,1.5974409108120948,1.0329516699343049,2.4815884369072774,0.368873,0,1,53.034234907155636,53.43905170597175,43.59058779443179,-9.443647112723845,60.74929878398849,7.310247078016744,22.32,2,0,43.59058779443179,-9.443647112723845,60.74929878398849,7.31024707

In [ ]:
np.unique(segment_id_borders,return_counts=True)

In [135]:
neuron_obj = du.fetch_neuron_obj_manual(segment_id=segment_id,
                                       border_validation=True)

Decompressing Neuron in minimal output mode...please wait


In [136]:
nviz.plot_axon(neuron_obj)


 Working on visualization type: mesh

 Working on visualization type: skeleton


# Way to Visualize the correct and incorrect branches

In [137]:
import validation_utils as vu
border_branches = vu.proofread_border_branches(neuron_obj,border_type="valid_to_invalid",
                            plot_border_branches=True)

Synapsse Results:
Total Number of Synapses = 3772
n_valid_syn_ids_presyn = 40
n_errored_syn_ids_presyn = 373
n_valid_syn_ids_postsyn = 3319
n_errored_syn_ids_postsyn = 40
10 boundary_branches : [ 35  36  38  62  63  64  67  94 111 113]

 Working on visualization type: mesh

 Working on visualization type: skeleton


# Will be able to zoom in on any of the border points to analyze

In [138]:
"""

Inputs: 
1) the border branch index you want to see
2) If want to print out the information about 

Psuedocode: 
1) find all the downstream branches
2) Find the shared skeleton endpoint for the downstream and border branch
--> turn the y value negative for zooming later

If want to print information:


"""

'\n\nInputs: \n1) the border branch index you want to see\n2) If want to print out the information about \n\nPsuedocode: \n1) find all the downstream branches\n2) Find the shared skeleton endpoint for the downstream and border branch\n--> turn the y value negative for zooming later\n\nIf want to print information:\n\n\n'

In [140]:
import axon_utils as au
neuron_obj = au.complete_axon_processing(neuron_obj,
                                        verbose=True)


Part 1: Axon like branchese 
{'L0': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17,
       18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 51]), 'L1': array([5]), 'L2': array([11]), 'L4': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
 

base_final_widths = [232.15526048 261.97593907 273.4573949 ]
overall_ais_width = 222.31414572958042
ais_width = 255.8628648190211
Adding the following branches as true axons: [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139.
 140. 141. 142.]
curr_soma_angle = 166.35
max_axon_angle = 171.56205865088413
Changing to a better axon candidate



  0%|          | 0/1 [00:00<?, ?it/s]

root_curr = [742948.9 748461.6 851662. ]
Time for preparing soma vertices and root: 0.000392913818359375
invalidation_d = 2000
cc_vertex_thresh = 10


100%|██████████| 1/1 [00:02<00:00,  2.30s/it]


meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 3.1548047065734863
branches_touching_root = [8]
combining close nodes
combine_threshold = 1300
length of Graph = 20288
After combining close endpoints max(kept_branches_idx) = 171, len(kept_branches_idx) = 156
empty_indices % = 0.0
 conflict_indices % = 0.02214013862822363
AFTER face_lookup_resolved_test
Decomposing first pass: 13.700690984725952
Only Using MeshParty Skeletonization and Mesh Correspondence
Another print
Divinding into MP and MAP pieces: 3.600120544433594e-05
Total time for MAP sublimb processing 3.5762786865234375e-06
---- Working on MP Decomposition #0 ----
Using Quicker soma_to_piece_touching_vertices because no MAP and only one sublimb_mesh piece 
MP filtering soma verts: 5.245208740234375e-05
Fixing Possible Soma Extension Branch for Sublimb 0
Total time for mesh KDTree = 0.003347158432006836
sbv[0].reshape(-1,3) = [[742948.9 748461.6 851662. ]]
closest_sk_pt_coord BEFORE = [743251.56236007 748961.8

In [141]:
import networkx_utils as xu
import neuron_utils as nru
import axon_utils as au

axon_limb_name = neuron_obj.axon_limb_name
curr_idx = 0
radius = 20000
border_color = "purple"
downstream_color = "red"
print_border_info = True

curr_border_idx = border_brnaches[curr_idx]

shared_skeleton_pt, downstream_branches = nru.skeleton_coordinate_connecting_to_downstream_branches(neuron_obj[ax_limb_name],
                                                                curr_border_idx,
                                                                        return_downstream_branches=True)



# plot the neuron at the current branching point

dict_list = [{axon_limb_name:[curr_border_idx]},
             {axon_limb_name:downstream_branches}]
color_list = [border_color,downstream_color]
color_dict = nviz.limb_branch_dicts_to_combined_color_dict(dict_list,
                                             color_list)

nviz.visualize_neuron(neuron_obj,
                  visualize_type=["mesh","skeleton"],
                  limb_branch_dict = {axon_limb_name:list(downstream_branches) + [curr_border_idx]},
                  mesh_color=color_dict,
                  skeleton_color=color_dict,
                  mesh_boutons=True,
                  mesh_web=True,
                  mesh_whole_neuron=True,
                                    )


if print_border_info:
    
    attr_dict = au.axon_branching_attributes(neuron_obj,
                                neuron_obj.axon_limb_idx,
                                curr_border_idx,
                                verbose=False)
    for k,v in attr_dict:
        print(f"{k}:{v}")
nviz.set_zoom(shared_skeleton_pt,
                  radius=3000,
             turn_axis_on=True)





 Working on visualization type: mesh

 Working on visualization type: skeleton


{'web_size_faces': 93, 'web_size_volume': 143702916.1653439, 'web_size_skeleton': 1725.6236448111838, 'web_size_ray_trace_percentile': 211.415177956351, 'web_bbox_ratios_max': 2.334004723393049, 'web_bbox_ratios_min': 1.1426624425432275, 'web_volume_ratio': 2.470780865280611, 'web_cdf': 0.317905, 'parent_n_large_boutons': 0, 'parent_n_boutons': 2, 'parent_no_bouton_median': 43.59058779443179, 'parent_no_spine_median_mesh_center': 60.74929878398849, 'child_no_bouton_median_min': 71.0792718236183, 'child_no_bouton_median_diff_min': 27.48868402918651, 'child_no_spine_median_mesh_center_min': 42.91526733380065, 'child_no_spine_median_mesh_center_diff_min': -17.834031450187844, 'child_angle_min': 32.24, 'child_n_boutons_min': 0, 'child_n_large_boutons_min': 0, 'child_no_bouton_median_max': 71.0792718236183, 'child_no_bouton_median_diff_max': 27.48868402918651, 'child_no_spine_median_mesh_center_max': 42.91526733380065, 'child_no_spine_median_mesh_center_diff_max': -17.834031450187844, 'chil

In [143]:
attr_dict = au.axon_branching_attributes(neuron_obj,
                            neuron_obj.axon_limb_idx,
                            curr_border_idx,
                            verbose=False)
for k,v in attr_dict.items():
    print(f"{k}:{v}")

web_size_faces:93
web_size_volume:143702916.1653439
web_size_skeleton:1725.6236448111838
web_size_ray_trace_percentile:211.4309228487844
web_bbox_ratios_max:2.334004723393049
web_bbox_ratios_min:1.1426624425432275
web_volume_ratio:2.470780865280611
web_cdf:0.317905
parent_n_large_boutons:0
parent_n_boutons:2
parent_no_bouton_median:43.59058779443179
parent_no_spine_median_mesh_center:60.74929878398849
child_no_bouton_median_min:71.0792718236183
child_no_bouton_median_diff_min:27.48868402918651
child_no_spine_median_mesh_center_min:42.91526733380065
child_no_spine_median_mesh_center_diff_min:-17.834031450187844
child_angle_min:32.24
child_n_boutons_min:0
child_n_large_boutons_min:0
child_no_bouton_median_max:71.0792718236183
child_no_bouton_median_diff_max:27.48868402918651
child_no_spine_median_mesh_center_max:42.91526733380065
child_no_spine_median_mesh_center_diff_max:-17.834031450187844
child_angle_max:132.35
child_n_boutons_max:5
child_n_large_boutons_max:2
sibling_angles_min:30.14

In [90]:
nru.skeleton_coordinate_connecting_to_downstream_branches(neuron_obj[ax_limb_name],
                                                         curr_border_idx,
                                                         verbose=True)

downstream_branches = [14 25 27 34]
shared_skeleton_pt= [728892.29090799 799220.1064664  885307.66648315]


array([728892.29090799, 799220.1064664 , 885307.66648315])

In [50]:

new_center

array([ 728892.29090799, -799220.1064664 ,  885307.66648315])

In [ ]:
"""
Purpose: To take the border 

"""

In [36]:
ax_limb_name = neuron_obj.axon_limb_name
axon_nx = neuron_obj[neuron_obj.axon_limb_name].concept_network_directional
axon_nx

array([728892.29090799, 799220.1064664 , 885307.66648315])